In [1]:
%matplotlib inline

import numpy as np
import random
import pandas as pd
import requests as rq
import sys
import io
import os
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import Callback


%load_ext tensorboard

Data Cleaning

In [2]:

##get data
##data is already from 2012-2020
data = pd.read_csv('data_file.csv')


##preprocessing

##filter to only patriots
data = data.loc[(data['home_team'] == 'NE') | (data['away_team'] == 'NE')]

##removes 
data = data.loc[(data['desc'].str.startswith('('))]
data = data[data['play_type'] != 'qb_kneel']
data = data[data['play_type'] != 'no_play']



 
##fixes playtype to be strictly pass or run 
##lables scrambles as passes
data.loc[data['pass']==1,'play_type'] = 'pass'
data.loc[data.rush==1,'play_type'] = 'run'


columnsKeep = ['play_id','game_id','season_type','week','posteam','posteam_type','defteam','side_of_field','yardline_100','half_seconds_remaining','home_team','away_team',
               'game_seconds_remaining','game_half','drive','down','time','yrdln','ydstogo','desc','play_type','yards_gained','shotgun','no_huddle','qb_dropback','qb_kneel',
               'qb_spike','qb_scramble','pass_length','run_location','run_gap','home_timeouts_remaining','away_timeouts_remaining','total_home_score','total_away_score',
               'passer_player_name','receiver_player_name','rusher_player_name','play_type_nfl','roof','surface','temp','wind','home_coach','away_coach','game_stadium',
               'success']

data = data[columnsKeep]


data.to_csv('cleaneddata.csv', index=False)


/var/folders/ys/6w09q22n4f73ds9jwmgbstjc0000gn/T/ipykernel_65887/1351778777.py:3: DtypeWarning: Columns (45,179,180,182,183,189,190,193,194,197,198,203,204,205,206,209,210,213,214,218,219,220,222,224,226,233,234,235,236,237,238,248,249,253,254,255,260,262,263,266,267,268,269,283,284,294,295,301) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data_file.csv')


In [3]:
features = data[['down','ydstogo','yardline_100', 'half_seconds_remaining', 'game_seconds_remaining', 'temp', 'wind']]
target = data['play_type'].map({'pass': 0, 'run': 1})

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['down','ydstogo','yardline_100', 'half_seconds_remaining', 'game_seconds_remaining', 'temp', 'wind'])
    ]
)

X_processed = preprocessor.fit_transform(features)

X_lstm = X_processed.reshape((X_processed.shape[0], 1, X_processed.shape[1]))
y = target.values

X_train, X_test, y_train, y_test = train_test_split(X_lstm, y, test_size=0.2, random_state=42)

model = Sequential([
    LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1, activation='softmax')
])

model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['accuracy']
)



##history = model.fit(X_train, y_train, epochs=50, validation_split=0.1, callbacks=[early_stopping], verbose=1)

##test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
##print(f'Test Accuracy: {test_acc}')



# Assuming you have already defined X_test and y_test
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int).reshape(-1)

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(X_train, y_train, epochs=50, validation_split=0.1, callbacks=[early_stopping], verbose=1)

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Accuracy: {test_acc}')


for i in range(10):
    actual = 'pass' if y_test[i] == 0 else 'run'
    predicted = 'pass' if predictions[i] == 0 else 'run'
    print(f'Actual: {actual}, Predicted: {predicted}')


  1/147 [..............................] - ETA: 23s

2024-04-11 18:26:55.070236: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-11 18:26:55.071448: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-11 18:26:55.071777: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

147/147 [==============================] - 0s 314us/step
Epoch 1/50


2024-04-11 18:26:55.410794: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-11 18:26:55.411645: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-11 18:26:55.412124: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

526/526 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.5545 - val_loss: nan - val_accuracy: 0.5717
Epoch 2/50
 97/526 [====>.........................] - ETA: 0s - loss: nan - accuracy: 0.5445

2024-04-11 18:26:56.532307: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-11 18:26:56.532934: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-11 18:26:56.533319: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

526/526 [==============================] - 0s 939us/step - loss: nan - accuracy: 0.5553 - val_loss: nan - val_accuracy: 0.5717
Epoch 3/50
526/526 [==============================] - 0s 879us/step - loss: nan - accuracy: 0.5553 - val_loss: nan - val_accuracy: 0.5717
Epoch 4/50
526/526 [==============================] - 0s 858us/step - loss: nan - accuracy: 0.5553 - val_loss: nan - val_accuracy: 0.5717
Epoch 5/50
526/526 [==============================] - 0s 856us/step - loss: nan - accuracy: 0.5553 - val_loss: nan - val_accuracy: 0.5717
Epoch 6/50
526/526 [==============================] - 0s 887us/step - loss: nan - accuracy: 0.5553 - val_loss: nan - val_accuracy: 0.5717
Epoch 7/50
526/526 [==============================] - 0s 902us/step - loss: nan - accuracy: 0.5553 - val_loss: nan - val_accuracy: 0.5717
Epoch 8/50
526/526 [==============================] - 0s 857us/step - loss: nan - accuracy: 0.5553 - val_loss: nan - val_accuracy: 0.5717
Epoch 9/50
526/526 [=========================